# NuScenes 单目点云生成教程

本notebook完整复刻了NuScenes点云生成流程，将forward方法拆分为多个独立步骤，便于理解和调试。

## 功能概述

1. 从深度图生成点云
2. 支持多种稀疏度级别（Drop90, Drop80, Drop50, Drop25, full）
3. 支持天空过滤和深度一致性检查
4. 支持边界框裁剪
5. 使用depth_utils进行深度图预处理（插值到原始尺寸）

## 使用说明

1. 按顺序执行所有单元格
2. 在"配置参数"单元格中修改场景路径和参数
3. 每个步骤单元格可以独立检查和调试
4. 查看"反直觉检查"单元格以发现潜在问题


## 1. 环境配置

安装和导入所有必要的依赖包。


In [1]:
# 安装依赖（如果需要）
# !pip install numpy imageio opencv-python open3d rich torch

import os
import sys
import numpy as np
import imageio.v2 as imageio
import cv2
import open3d as o3d
from typing import Literal
from rich.console import Console
from copy import deepcopy

# 添加项目路径以导入模块
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

# 添加depth_utils路径
depth_utils_path = os.path.join(
    project_root, 
    'third_party/EVolSplat/preprocess/metric3d/mono/tools'
)
sys.path.insert(0, depth_utils_path)

# 初始化Console
CONSOLE = Console(width=120)

# 检查关键依赖
try:
    import torch
    CONSOLE.log("[green]✓ PyTorch installed[/green]")
except ImportError:
    CONSOLE.log("[red]✗ PyTorch not found. Please install: pip install torch[/red]")

try:
    from depth_utils import process_depth_for_use, load_depth_with_metadata
    CONSOLE.log("[green]✓ depth_utils imported successfully[/green]")
except ImportError as e:
    CONSOLE.log(f"[red]✗ Failed to import depth_utils: {e}[/red]")
    CONSOLE.log(f"[yellow]Make sure depth_utils.py is at: {depth_utils_path}[/yellow]")

CONSOLE.log("[green]Environment setup completed![/green]")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[19:52:25] ✓ PyTorch installed                                                                           ]8;id=919860;file:///tmp/ipykernel_96148/479480832.py\479480832.py]8;;\:]8;id=61358;file:///tmp/ipykernel_96148/479480832.py#31\31]8;;\

           ✓ depth_utils imported successfully                                                           ]8;id=910034;file:///tmp/ipykernel_96148/479480832.py\479480832.py]8;;\:]8;id=178363;file:///tmp/ipykernel_96148/479480832.py#37\37]8;;\

           Environment setup completed!                                                                  ]8;id=552433;file:///tmp/ipykernel_96148/479480832.py\479480832.py]8;;\:]8;id=109545;file:///tmp/ipykernel_96148/479480832.py#42\42]8;;\

## 2. 配置参数

修改以下参数以适配你的场景。


In [2]:
# ========== 场景配置 ==========
# 场景目录路径（例如: /path/to/trainval/000）
scene_dir = "/root/autodl-tmp/nuScenes/preprocess/trainval/000"

# ========== 相机配置 ==========
# 选择要使用的相机ID列表，例如 [0] 表示只使用前置摄像头，[0, 1, 2, 3, 4, 5] 表示使用所有相机
# 注意：必须与深度图中实际存在的相机ID匹配
chosen_cam_id = [0]  # 如果深度图只有000_0, 001_0等，则设置为[0]

# ========== 稀疏度配置 ==========
# 可选值: 'Drop90', 'Drop80', 'Drop50', 'Drop25', 'full'
sparsity = 'Drop50'

# ========== 帧范围配置 ==========
frame_start = 0  # 起始帧索引
num_frames = 20  # 要处理的帧数（None表示处理所有帧）

# ========== 功能开关 ==========
filter_sky = True  # 是否过滤天空区域
depth_consistency = True  # 是否进行深度一致性检查
use_bbx = True  # 是否使用边界框裁剪

# ========== 下采样配置 ==========
downscale = 2  # 点云生成时的下采样倍数

# ========== 保存配置 ==========
save_dir = None  # 保存目录名称（None则使用sparsity值）

# 显示配置
CONSOLE.log(f"[cyan]Configuration:[/cyan]")
CONSOLE.log(f"  Scene directory: {scene_dir}")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")
CONSOLE.log(f"  Sparsity: {sparsity}")
CONSOLE.log(f"  Frame range: {frame_start} to {frame_start + (num_frames if num_frames else 'all')}")
CONSOLE.log(f"  Filter sky: {filter_sky}")
CONSOLE.log(f"  Depth consistency: {depth_consistency}")
CONSOLE.log(f"  Use bounding box: {use_bbx}")
CONSOLE.log(f"  Downscale: {downscale}")


[19:52:27] Configuration:                                                                               ]8;id=811958;file:///tmp/ipykernel_96148/3050187033.py\3050187033.py]8;;\:]8;id=954879;file:///tmp/ipykernel_96148/3050187033.py#30\30]8;;\

             Scene directory: /root/autodl-tmp/nuScenes/preprocess/trainval/000                         ]8;id=663817;file:///tmp/ipykernel_96148/3050187033.py\3050187033.py]8;;\:]8;id=832856;file:///tmp/ipykernel_96148/3050187033.py#31\31]8;;\

             Chosen camera IDs: [0]                                                                     ]8;id=860912;file:///tmp/ipykernel_96148/3050187033.py\3050187033.py]8;;\:]8;id=698972;file:///tmp/ipykernel_96148/3050187033.py#32\32]8;;\

             Sparsity: Drop50                                                                           ]8;id=522324;file:///tmp/ipykernel_96148/3050187033.py\3050187033.py]8;;\:]8;id=878799;file:///tmp/ipykernel_96148/3050187033.py#33\33]8;;\

             Frame range: 0 to 20                                                                       ]8;id=171005;file:///tmp/ipykernel_96148/3050187033.py\3050187033.py]8;;\:]8;id=225236;file:///tmp/ipykernel_96148/3050187033.py#34\34]8;;\

             Filter sky: True                                                                           ]8;id=243461;file:///tmp/ipykernel_96148/3050187033.py\3050187033.py]8;;\:]8;id=481725;file:///tmp/ipykernel_96148/3050187033.py#35\35]8;;\

             Depth consistency: True                                                                    ]8;id=920598;file:///tmp/ipykernel_96148/3050187033.py\3050187033.py]8;;\:]8;id=79700;file:///tmp/ipykernel_96148/3050187033.py#36\36]8;;\

             Use bounding box: True                                                                     ]8;id=574387;file:///tmp/ipykernel_96148/3050187033.py\3050187033.py]8;;\:]8;id=891922;file:///tmp/ipykernel_96148/3050187033.py#37\37]8;;\

             Downscale: 2                                                                               ]8;id=146284;file:///tmp/ipykernel_96148/3050187033.py\3050187033.py]8;;\:]8;id=948315;file:///tmp/ipykernel_96148/3050187033.py#38\38]8;;\

## 3. NuScenes场景文件夹结构

### 目录结构

```
scene_dir/ (例如: /path/to/trainval/000)
├── images/                    # RGB图像目录
│   ├── 000_0.jpg (或 .png)   # 格式: {frame_idx:03d}_{cam_id}.jpg
│   ├── 000_1.jpg
│   ├── 001_0.jpg
│   └── ...
├── depth/                     # 深度图目录
│   ├── 000_0.npy             # 格式: {frame_idx:03d}_{cam_id}.npy
│   ├── 000_0_meta.npz        # 元数据文件（包含原始尺寸、内参等）
│   ├── 000_1.npy
│   ├── 000_1_meta.npz
│   └── ...
├── extrinsics/                # 相机外参（cam_to_world变换矩阵）
│   ├── 000_0.txt             # 格式: {frame_idx:03d}_{cam_id}.txt (4x4矩阵)
│   ├── 000_1.txt
│   └── ...
├── intrinsics/                # 相机内参（每个相机固定）
│   ├── 0.txt                 # 格式: {cam_id}.txt (fx, fy, cx, cy)
│   ├── 1.txt
│   └── ...
├── sky_masks/                 # 天空掩码（可选）
│   ├── 000_0.png             # 格式: {frame_idx:03d}_{cam_id}.png (0=sky, 255=non-sky)
│   └── ...
├── transforms.json           # 场景变换信息（可选）
└── [sparsity_dir]/           # 点云输出目录（如Drop50, Drop80等）
    └── 0.ply                 # 生成的点云文件
```

### 文件命名规则

- **图像**: `{frame_idx:03d}_{cam_id}.jpg` 或 `.png`
- **深度图**: `{frame_idx:03d}_{cam_id}.npy` (对应元数据: `{frame_idx:03d}_{cam_id}_meta.npz`)
- **外参**: `{frame_idx:03d}_{cam_id}.txt` (4x4矩阵，每行一个值)
- **内参**: `{cam_id}.txt` (fx, fy, cx, cy，每行一个值)

### 重要说明

1. **深度图尺寸**: 深度图可能不是原始图像尺寸（模型输出去除了padding），需要使用`depth_utils.process_depth_for_use()`进行插值
2. **元数据文件**: 每个深度图都有对应的`_meta.npz`文件，包含原始尺寸、内参等信息
3. **坐标系统**: 外参是cam_to_world变换矩阵，已经对齐到第一帧第一相机


## 4. 辅助函数

定义读取图像尺寸和位姿/内参的函数。


In [3]:
def get_image_dimensions(scene_dir: str):
    """
    从场景目录获取图像尺寸。
    
    Args:
        scene_dir: 场景目录路径
        
    Returns:
        (H, W): 图像高度和宽度
    """
    images_dir = os.path.join(scene_dir, 'images')
    if not os.path.exists(images_dir):
        raise ValueError(f"Images directory not found: {images_dir}")
    
    image_files = [f for f in os.listdir(images_dir) 
                   if f.endswith('.jpg') or f.endswith('.png')]
    if len(image_files) == 0:
        raise ValueError(f"No image files found in {images_dir}")
    
    # 读取第一张图像获取尺寸
    first_image_path = os.path.join(images_dir, sorted(image_files)[0])
    img = imageio.imread(first_image_path)
    H, W = img.shape[0], img.shape[1]
    
    return H, W


def read_poses_intrinsics_direct(scene_dir: str, frame_start: int = 0, num_frames: int = None):
    """
    直接从场景目录读取poses和intrinsics，按照NuScenes场景文件夹的预定结构。
    
    Args:
        scene_dir: 场景目录路径
        frame_start: 起始帧索引（默认0）
        num_frames: 要读取的帧数（None表示读取所有帧）
        
    Returns:
        poses: Array of poses (num_frames * num_cameras, 4, 4)
        intrinsics: Array of intrinsics (num_frames * num_cameras, 4, 4)
        info: Tuple (H, W) - 图像尺寸
    """
    extrinsics_dir = os.path.join(scene_dir, 'extrinsics')
    intrinsics_dir = os.path.join(scene_dir, 'intrinsics')
    images_dir = os.path.join(scene_dir, 'images')
    
    if not os.path.exists(extrinsics_dir):
        raise ValueError(f"Extrinsics directory not found: {extrinsics_dir}")
    if not os.path.exists(intrinsics_dir):
        raise ValueError(f"Intrinsics directory not found: {intrinsics_dir}")
    if not os.path.exists(images_dir):
        raise ValueError(f"Images directory not found: {images_dir}")
    
    # 获取所有帧索引和相机ID
    extrinsic_files = sorted([f for f in os.listdir(extrinsics_dir) if f.endswith('.txt')])
    frame_cam_pairs = []
    for f in extrinsic_files:
        try:
            parts = f.replace('.txt', '').split('_')
            if len(parts) >= 2:
                frame_idx = int(parts[0])
                cam_id = int(parts[1])
                frame_cam_pairs.append((frame_idx, cam_id))
        except (ValueError, IndexError):
            continue
    
    # 过滤帧范围
    if num_frames is not None:
        frame_end = frame_start + num_frames
        frame_cam_pairs = [(f, c) for f, c in frame_cam_pairs if frame_start <= f < frame_end]
    else:
        frame_cam_pairs = [(f, c) for f, c in frame_cam_pairs if f >= frame_start]
    
    # 按frame_idx和cam_id排序
    frame_cam_pairs.sort(key=lambda x: (x[0], x[1]))
    
    # 加载第一帧第一相机的pose用于对齐
    camera_front_start = None
    first_frame_cam = None
    for frame_idx, cam_id in frame_cam_pairs:
        if cam_id == 0:  # 使用相机0作为参考
            first_extrinsic_file = os.path.join(extrinsics_dir, f'{frame_idx:03d}_{cam_id}.txt')
            if os.path.exists(first_extrinsic_file):
                camera_front_start = np.loadtxt(first_extrinsic_file)
                first_frame_cam = (frame_idx, cam_id)
                break
    
    if camera_front_start is None:
        # Fallback: 使用第一个可用的extrinsic
        if len(frame_cam_pairs) > 0:
            first_frame_cam = frame_cam_pairs[0]
            first_extrinsic_file = os.path.join(extrinsics_dir, f'{first_frame_cam[0]:03d}_{first_frame_cam[1]}.txt')
            if os.path.exists(first_extrinsic_file):
                camera_front_start = np.loadtxt(first_extrinsic_file)
                CONSOLE.log(f"Warning: Using frame {first_frame_cam[0]} camera {first_frame_cam[1]} for alignment")
    
    # 读取所有poses和intrinsics
    poses = []
    intrinsics = []
    
    # 获取相机ID列表
    cam_ids = sorted(set(cam_id for _, cam_id in frame_cam_pairs))
    
    # 读取每个相机的内参（固定值）
    cam_intrinsics_dict = {}
    for cam_id in cam_ids:
        intrinsic_file = os.path.join(intrinsics_dir, f'{cam_id}.txt')
        if os.path.exists(intrinsic_file):
            intrinsic_data = np.loadtxt(intrinsic_file)
            fx, fy, cx, cy = intrinsic_data[0], intrinsic_data[1], intrinsic_data[2], intrinsic_data[3]
            # 转换为4x4矩阵
            cam_intrinsic = np.eye(4)
            cam_intrinsic[0, 0] = fx
            cam_intrinsic[1, 1] = fy
            cam_intrinsic[0, 2] = cx
            cam_intrinsic[1, 2] = cy
            cam_intrinsics_dict[cam_id] = cam_intrinsic
        else:
            CONSOLE.log(f"Warning: Intrinsic file not found: {intrinsic_file}")
    
    # 读取每个frame_cam对的pose
    for frame_idx, cam_id in frame_cam_pairs:
        extrinsic_file = os.path.join(extrinsics_dir, f'{frame_idx:03d}_{cam_id}.txt')
        if not os.path.exists(extrinsic_file):
            CONSOLE.log(f"Warning: Extrinsic file not found: {extrinsic_file}, skipping")
            continue
        
        # 加载外参（cam_to_world）
        cam2world = np.loadtxt(extrinsic_file)
        
        # 对齐到第一帧第一相机（如果可用）
        if camera_front_start is not None:
            cam2world = np.linalg.inv(camera_front_start) @ cam2world
        
        # OPENCV2DATASET是单位矩阵，不需要转换
        OPENCV2DATASET = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        cam2world = cam2world @ OPENCV2DATASET
        
        poses.append(cam2world)
        
        # 添加对应的内参
        if cam_id in cam_intrinsics_dict:
            intrinsics.append(cam_intrinsics_dict[cam_id])
        else:
            # 如果内参不存在，使用单位矩阵
            intrinsics.append(np.eye(4))
    
    # 获取图像尺寸
    H, W = get_image_dimensions(scene_dir)
    
    return np.array(poses), np.array(intrinsics), (H, W)

CONSOLE.log("[green]Helper functions defined![/green]")


[19:52:30] Helper functions defined!                                                                   ]8;id=454141;file:///tmp/ipykernel_96148/4137707433.py\4137707433.py]8;;\:]8;id=337868;file:///tmp/ipykernel_96148/4137707433.py#152\152]8;;\

In [4]:
# 导入depth_utils
try:
    from depth_utils import process_depth_for_use, load_depth_with_metadata
    CONSOLE.log("[green]✓ depth_utils imported successfully[/green]")
    
    # 示例：如何加载和预处理深度图
    # depth_file = os.path.join(scene_dir, 'depth', '000_0.npy')
    # depth, metadata = process_depth_for_use(depth_file, target_shape=(H, W))
    # print(f"Depth shape: {depth.shape}")
    # print(f"Original shape: {metadata['ori_shape']}")
    
except ImportError as e:
    CONSOLE.log(f"[red]✗ Failed to import depth_utils: {e}[/red]")
    CONSOLE.log("[yellow]Make sure depth_utils.py is in the correct path[/yellow]")


[19:52:31] ✓ depth_utils imported successfully                                                           ]8;id=347513;file:///tmp/ipykernel_96148/3421549315.py\3421549315.py]8;;\:]8;id=357886;file:///tmp/ipykernel_96148/3421549315.py#4\4]8;;\

## 6. NuScenesPCDGenerator类定义

定义点云生成器的辅助方法（不包含forward方法，forward将被拆分为多个步骤）。


In [5]:
# Default bounding box for nuScenes (similar to Waymo, can be customized)
X_MIN, X_MAX = -20, 20
Y_MIN, Y_MAX = -20, 4.8
Z_MIN, Z_MAX = -20, 70

class NuScenesPCDGenerator():
    """Point cloud generator for nuScenes dataset."""
    sparsity: Literal['Drop90','Drop50',"Drop80","Drop25","full"] = "full"
    use_bbx: bool = True

    def __init__(self, spars='full', save_dir="Drop50", frame_start=0, 
                 filer_sky=True, depth_cosistency=True) -> None:
        self.sparsity = spars
        self.save_dir = save_dir
        self.dir_name = None
        self.frame_start = frame_start
        self.filter_sky = filer_sky
        self.depth_cosistency = depth_cosistency

    def get_bbx(self):
        """Get bounding box for point cloud filtering."""
        return np.array([X_MIN, Y_MIN, Z_MIN]), np.array([X_MAX, Y_MAX, Z_MAX])

    def crop_pointcloud(self, bbx_min, bbx_max, points, color):
        """Crop point cloud to bounding box."""
        mask = (points[:, 0] > bbx_min[0]) & (points[:, 0] < bbx_max[0]) & \
            (points[:, 1] > bbx_min[1]) & (points[:, 1] < bbx_max[1]) & \
            (points[:, 2] > bbx_min[2]) & (points[:, 2] < bbx_max[2] + 50)  # Extended Z for background
        
        return points[mask], color[mask]
    
    def split_pointcloud(self, bbx_min, bbx_max, points, color):
        """Split point cloud into inside and outside bounding box."""
        mask = (points[:, 0] > bbx_min[0]) & (points[:, 0] < bbx_max[0]) & \
            (points[:, 1] > bbx_min[1]) & (points[:, 1] < bbx_max[1]) & \
            (points[:, 2] > bbx_min[2]) & (points[:, 2] < bbx_max[2])
        
        inside_pnt, inside_rgb = points[mask], color[mask]
        outside_pnt, outside_rgb = points[~mask], color[~mask]
        return inside_pnt, inside_rgb, outside_pnt, outside_rgb

    def depth_cosistency_check(self, depth_files, H, W):
        """Check depth consistency between consecutive frames."""
        depth_masks = []
        print("Depth Consistency Check!")
        
        for i, file_name in enumerate(depth_files):
            depth_file = os.path.join(self.depth_dir, file_name)
            try:
                # 使用depth_utils加载深度图
                depth, metadata = process_depth_for_use(depth_file, target_shape=(H, W))
            except Exception as e:
                CONSOLE.log(f"Warning: Failed to load depth {file_name}: {e}")
                depth_masks.append(np.ones((H, W)))
                continue

            # Assume the first depth frame is correct
            if i == 0:
                last_depth = deepcopy(depth)
                depth_masks.append(np.ones((H, W)))
                continue

            try:
                c2w = self.c2w[i]
                last_c2w = self.c2w[i-1]
                K = self.intri[i]
            except IndexError as e:
                depth_masks.append(np.ones((H, W)))
                continue

            # Unproject pointcloud
            x = np.arange(0, depth.shape[1])
            y = np.arange(0, depth.shape[0])
            xx, yy = np.meshgrid(x, y)
            pixels = np.vstack((xx.ravel(), yy.ravel())).T.reshape(-1, 2)

            # Unproject depth to pointcloud
            cx, cy = K[0, 2], K[1, 2]
            fx, fy = K[0, 0], K[1, 1]
            
            x_cam = (pixels[..., 0] - cx) * depth.reshape(-1) / fx
            y_cam = (pixels[..., 1] - cy) * depth.reshape(-1) / fy
            z_cam = depth.reshape(-1)
            coordinates = np.stack([x_cam, y_cam, z_cam], axis=1)

            depth_mask = self.depth_projection_check(
                coordinates=coordinates, pixels=pixels, 
                last_c2w=last_c2w, c2w=c2w, 
                last_depth=last_depth, depth=depth, K=K
            )
            depth_masks.append(depth_mask)

            # Update status
            last_depth = deepcopy(depth)

        return depth_masks

    def depth_projection_check(self, coordinates, pixels, last_c2w, c2w, last_depth, depth, K):
        """Check depth consistency by projecting current points to previous frame."""
        H, W = last_depth.shape[:2]
        cx, cy = K[0, 2], K[1, 2]
        fx, fy = K[0, 0], K[1, 1]

        trans_mat = np.dot(np.linalg.inv(last_c2w), c2w)
        coordinates_homo = np.column_stack((coordinates.reshape(-1, 3), np.ones(len(coordinates))))
        last_coordinates = np.dot(trans_mat, coordinates_homo.T).T
        
        # Project to previous frame
        last_x = (fx * last_coordinates[:, 0] + cx * last_coordinates[:, 2]) / last_coordinates[:, 2]
        last_y = (fy * last_coordinates[:, 1] + cy * last_coordinates[:, 2]) / last_coordinates[:, 2]
        last_pixels = np.vstack((last_x, last_y)).T.reshape(-1, 2).astype(np.int32)

        # Swap pixel coordinates (row, col) <-> (x, y)
        pixels_swapped = pixels.copy()
        pixels_swapped[:, [0, 1]] = pixels_swapped[:, [1, 0]]
        last_pixels_swapped = last_pixels.copy()
        last_pixels_swapped[:, [0, 1]] = last_pixels_swapped[:, [1, 0]]

        depth_mask = np.ones(depth.shape[0] * depth.shape[1])

        # Reprojection location must be in image plane [0,H] [0,W]
        valid_mask_00 = np.logical_and((last_pixels_swapped[:, 0] < H), (last_pixels_swapped[:, 1] < W))
        valid_mask_01 = np.logical_and((last_pixels_swapped[:, 0] > 0), (last_pixels_swapped[:, 1] > 0))
        valid_mask = np.logical_and(valid_mask_00, valid_mask_01)

        depth_diff = np.abs(
            depth[pixels_swapped[valid_mask, 0], pixels_swapped[valid_mask, 1]] - 
            last_depth[last_pixels_swapped[valid_mask, 0], last_pixels_swapped[valid_mask, 1]]
        )
        depth_mask[valid_mask] = np.where(depth_diff < depth_diff.mean(), 1, 0)
        depth_mask = depth_mask.reshape(*depth.shape)

        return depth_mask

CONSOLE.log("[green]NuScenesPCDGenerator class defined![/green]")


[19:52:34] NuScenesPCDGenerator class defined!                                                         ]8;id=867637;file:///tmp/ipykernel_96148/3590799288.py\3590799288.py]8;;\:]8;id=691312;file:///tmp/ipykernel_96148/3590799288.py#135\135]8;;\

In [6]:
# 验证场景目录
if not os.path.exists(scene_dir):
    raise ValueError(f"Scene directory not found: {scene_dir}")

# 检查必要的目录和文件
required_dirs = ['images', 'extrinsics', 'intrinsics', 'depth']
for dir_name in required_dirs:
    dir_path = os.path.join(scene_dir, dir_name)
    if not os.path.exists(dir_path):
        raise ValueError(f"Required directory not found: {dir_path}")
    CONSOLE.log(f"[green]✓ Found {dir_name}/[/green]")

# 检查天空掩码目录（如果启用天空过滤）
if filter_sky:
    sky_masks_dir = os.path.join(scene_dir, 'sky_masks')
    if not os.path.exists(sky_masks_dir):
        CONSOLE.log(f"[yellow]Warning: Sky masks directory not found: {sky_masks_dir}[/yellow]")
        CONSOLE.log("[yellow]Sky filtering will be disabled. Run gen_nuscenes_depth_mask.py --gen_sky_mask first.[/yellow]")
        filter_sky = False
    else:
        CONSOLE.log(f"[green]✓ Found sky_masks/[/green]")

# 验证深度文件存在
depth_dir = os.path.join(scene_dir, 'depth')
all_depth_files = [f for f in os.listdir(depth_dir) if f.endswith('.npy') and not f.endswith('_meta.npz')]
if len(all_depth_files) == 0:
    raise ValueError(f"No depth files found in {depth_dir}")
CONSOLE.log(f"[green]✓ Found {len(all_depth_files)} total depth files[/green]")

# 检查深度文件中的相机ID
depth_cam_ids = set()
for f in all_depth_files:
    try:
        parts = f.replace('.npy', '').split('_')
        if len(parts) >= 2:
            depth_cam_ids.add(int(parts[1]))
    except:
        continue

CONSOLE.log(f"[cyan]Camera ID analysis:[/cyan]")
CONSOLE.log(f"  Camera IDs found in depth files: {sorted(depth_cam_ids)}")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")

# 检查chosen_cam_id是否在深度文件中存在
missing_cams = set(chosen_cam_id) - depth_cam_ids
if missing_cams:
    CONSOLE.log(f"[yellow]⚠ Warning: Some chosen_cam_id ({missing_cams}) not found in depth files[/yellow]")
    CONSOLE.log(f"[yellow]  Available camera IDs: {sorted(depth_cam_ids)}[/yellow]")

# 检查是否有深度文件使用了不在chosen_cam_id中的相机
unexpected_cams = depth_cam_ids - set(chosen_cam_id)
if unexpected_cams:
    CONSOLE.log(f"[cyan]  Note: Depth files also contain cameras {sorted(unexpected_cams)} (will be filtered)[/cyan]")

# 检查深度图元数据文件
meta_files = [f for f in os.listdir(depth_dir) if f.endswith('_meta.npz')]
CONSOLE.log(f"[green]✓ Found {len(meta_files)} metadata files[/green]")
if len(meta_files) < len(all_depth_files):
    CONSOLE.log(f"[yellow]Warning: Some depth files may be missing metadata files[/yellow]")

CONSOLE.log("[green]Data validation completed![/green]")


[19:52:40] ✓ Found images/                                                                              ]8;id=429240;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=533052;file:///tmp/ipykernel_96148/2378628925.py#11\11]8;;\

           ✓ Found extrinsics/                                                                          ]8;id=227653;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=904184;file:///tmp/ipykernel_96148/2378628925.py#11\11]8;;\

           ✓ Found intrinsics/                                                                          ]8;id=729210;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=687698;file:///tmp/ipykernel_96148/2378628925.py#11\11]8;;\

           ✓ Found depth/                                                                               ]8;id=152038;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=267615;file:///tmp/ipykernel_96148/2378628925.py#11\11]8;;\

           ✓ Found sky_masks/                                                                           ]8;id=501523;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=751517;file:///tmp/ipykernel_96148/2378628925.py#21\21]8;;\

...

           Camera ID analysis:                                                                          ]8;id=864703;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=900898;file:///tmp/ipykernel_96148/2378628925.py#40\40]8;;\

             Camera IDs found in depth files: [0]                                                       ]8;id=29867;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=732904;file:///tmp/ipykernel_96148/2378628925.py#41\41]8;;\

             Chosen camera IDs: [0]                                                                     ]8;id=579519;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=570780;file:///tmp/ipykernel_96148/2378628925.py#42\42]8;;\

           ✓ Found 196 metadata files                                                                   ]8;id=812412;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=342140;file:///tmp/ipykernel_96148/2378628925.py#57\57]8;;\

           Data validation completed!                                                                   ]8;id=219628;file:///tmp/ipykernel_96148/2378628925.py\2378628925.py]8;;\:]8;id=623088;file:///tmp/ipykernel_96148/2378628925.py#61\61]8;;\

## 步骤1: 初始化与目录设置

设置目录路径、获取图像尺寸、初始化点云生成器。


In [7]:
# 读取poses和intrinsics
# 注意：这里读取所有相机的poses/intrinsics作为查找表
# 后续步骤会根据chosen_cam_id和深度文件来选择对应的poses
CONSOLE.log(f"Reading poses and intrinsics from scene: {scene_dir}")
poses, intrinsics, info = read_poses_intrinsics_direct(
    scene_dir=scene_dir,
    frame_start=frame_start,
    num_frames=num_frames
)
H, W = info[0], info[1]
CONSOLE.log(f"[green]Loaded {len(poses)} poses and intrinsics (H={H}, W={W})[/green]")
CONSOLE.log(f"[cyan]Note: These poses/intrinsics include all cameras. We will filter by chosen_cam_id={chosen_cam_id} in later steps.[/cyan]")

# 创建点云生成器
save_dir_name = save_dir if save_dir is not None else sparsity
pcd_generator = NuScenesPCDGenerator(
    spars=sparsity,
    save_dir=save_dir_name,
    frame_start=frame_start,
    filer_sky=filter_sky,
    depth_cosistency=depth_consistency
)
pcd_generator.use_bbx = use_bbx

# 设置目录路径
pcd_generator.dir_name = scene_dir
pcd_generator.depth_dir = os.path.join(scene_dir, 'depth')
pcd_generator.H, pcd_generator.W = H, W

CONSOLE.log("[green]Step 1: Initialization completed![/green]")
CONSOLE.log(f"  Image dimensions: H={H}, W={W}")
CONSOLE.log(f"  Total poses/intrinsics (all cameras): {len(poses)}")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")


[19:52:43] Reading poses and intrinsics from scene: /root/autodl-tmp/nuScenes/preprocess/trainval/000    ]8;id=883519;file:///tmp/ipykernel_96148/1290011656.py\1290011656.py]8;;\:]8;id=202172;file:///tmp/ipykernel_96148/1290011656.py#4\4]8;;\

           Loaded 120 poses and intrinsics (H=900, W=1600)                                              ]8;id=526487;file:///tmp/ipykernel_96148/1290011656.py\1290011656.py]8;;\:]8;id=282131;file:///tmp/ipykernel_96148/1290011656.py#11\11]8;;\

           Note: These poses/intrinsics include all cameras. We will filter by chosen_cam_id=[0] in     ]8;id=78512;file:///tmp/ipykernel_96148/1290011656.py\1290011656.py]8;;\:]8;id=69281;file:///tmp/ipykernel_96148/1290011656.py#12\12]8;;\
           later steps.                                                                                                 

           Step 1: Initialization completed!                                                            ]8;id=936588;file:///tmp/ipykernel_96148/1290011656.py\1290011656.py]8;;\:]8;id=794458;file:///tmp/ipykernel_96148/1290011656.py#30\30]8;;\

             Image dimensions: H=900, W=1600                                                            ]8;id=259882;file:///tmp/ipykernel_96148/1290011656.py\1290011656.py]8;;\:]8;id=362576;file:///tmp/ipykernel_96148/1290011656.py#31\31]8;;\

             Total poses/intrinsics (all cameras): 120                                                  ]8;id=264221;file:///tmp/ipykernel_96148/1290011656.py\1290011656.py]8;;\:]8;id=596608;file:///tmp/ipykernel_96148/1290011656.py#32\32]8;;\

             Chosen camera IDs: [0]                                                                     ]8;id=988313;file:///tmp/ipykernel_96148/1290011656.py\1290011656.py]8;;\:]8;id=279538;file:///tmp/ipykernel_96148/1290011656.py#33\33]8;;\

## 步骤2: 加载和分组深度文件

加载所有深度文件并按帧索引分组，为稀疏度过滤做准备。


In [8]:
# 加载所有深度文件（排除元数据文件）
all_depth_files = sorted([
    f for f in os.listdir(pcd_generator.depth_dir) 
    if f.endswith('.npy') and not f.endswith('_meta.npz')
])

CONSOLE.log(f"Found {len(all_depth_files)} total depth files")

# 根据chosen_cam_id过滤深度文件
# 深度文件格式: {frame_idx:03d}_{cam_id}.npy
filtered_depth_files = []
for file_name in all_depth_files:
    try:
        # 解析相机ID: {frame_idx:03d}_{cam_id}.npy
        parts = file_name.replace('.npy', '').split('_')
        if len(parts) >= 2:
            cam_id = int(parts[1])
            if cam_id in chosen_cam_id:
                filtered_depth_files.append(file_name)
    except (ValueError, IndexError):
        continue

CONSOLE.log(f"[cyan]Camera filtering:[/cyan]")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")
CONSOLE.log(f"  Filtered depth files: {len(filtered_depth_files)} (from {len(all_depth_files)} total)")

# 按帧索引分组（只包含chosen_cam_id中的相机）
frame_groups = {}
for file_name in filtered_depth_files:
    try:
        # 解析帧索引: {frame_idx:03d}_{cam_id}.npy
        frame_idx = int(file_name.split('_')[0])
        if frame_idx not in frame_groups:
            frame_groups[frame_idx] = []
        frame_groups[frame_idx].append(file_name)
    except (ValueError, IndexError) as e:
        CONSOLE.log(f"Warning: Cannot parse filename {file_name}, skipping")
        continue

# 显示分组统计
sorted_frame_indices = sorted(frame_groups.keys())
CONSOLE.log(f"[cyan]Frame grouping statistics:[/cyan]")
CONSOLE.log(f"  Total frames: {len(sorted_frame_indices)}")
if len(sorted_frame_indices) > 0:
    CONSOLE.log(f"  Frame range: {sorted_frame_indices[0]} to {sorted_frame_indices[-1]}")
    cameras_per_frame = len(frame_groups[sorted_frame_indices[0]])
    CONSOLE.log(f"  Cameras per frame: {cameras_per_frame} (should match chosen_cam_id: {len(chosen_cam_id)})")

CONSOLE.log("[green]Step 2: Depth file loading and grouping completed![/green]")


[19:52:46] Found 196 total depth files                                                                    ]8;id=517114;file:///tmp/ipykernel_96148/278330606.py\278330606.py]8;;\:]8;id=488797;file:///tmp/ipykernel_96148/278330606.py#7\7]8;;\

           Camera filtering:                                                                             ]8;id=8725;file:///tmp/ipykernel_96148/278330606.py\278330606.py]8;;\:]8;id=559223;file:///tmp/ipykernel_96148/278330606.py#23\23]8;;\

             Chosen camera IDs: [0]                                                                      ]8;id=723203;file:///tmp/ipykernel_96148/278330606.py\278330606.py]8;;\:]8;id=937572;file:///tmp/ipykernel_96148/278330606.py#24\24]8;;\

             Filtered depth files: 196 (from 196 total)                                                  ]8;id=842045;file:///tmp/ipykernel_96148/278330606.py\278330606.py]8;;\:]8;id=985401;file:///tmp/ipykernel_96148/278330606.py#25\25]8;;\

           Frame grouping statistics:                                                                    ]8;id=529963;file:///tmp/ipykernel_96148/278330606.py\278330606.py]8;;\:]8;id=235540;file:///tmp/ipykernel_96148/278330606.py#42\42]8;;\

             Total frames: 196                                                                           ]8;id=950620;file:///tmp/ipykernel_96148/278330606.py\278330606.py]8;;\:]8;id=15646;file:///tmp/ipykernel_96148/278330606.py#43\43]8;;\

             Frame range: 0 to 195                                                                       ]8;id=761889;file:///tmp/ipykernel_96148/278330606.py\278330606.py]8;;\:]8;id=263403;file:///tmp/ipykernel_96148/278330606.py#45\45]8;;\

             Cameras per frame: 1 (should match chosen_cam_id: 1)                                        ]8;id=388266;file:///tmp/ipykernel_96148/278330606.py\278330606.py]8;;\:]8;id=722391;file:///tmp/ipykernel_96148/278330606.py#47\47]8;;\

           Step 2: Depth file loading and grouping completed!                                            ]8;id=523587;file:///tmp/ipykernel_96148/278330606.py\278330606.py]8;;\:]8;id=639929;file:///tmp/ipykernel_96148/278330606.py#49\49]8;;\

## 步骤3: 稀疏度过滤

根据稀疏度级别过滤帧，减少点云密度。


In [9]:
# 根据稀疏度级别过滤帧
selected_frames = []
for frame_pos, frame_idx in enumerate(sorted_frame_indices):
    # 应用稀疏度过滤
    if sparsity == "Drop50":
        if frame_pos % 4 == 2 or frame_pos % 4 == 3:
            continue  # 保留50%的帧
    elif sparsity == 'Drop80':
        if frame_pos % 5 != 0:  # 保留20%的帧
            continue 
    elif sparsity == 'Drop25':
        if frame_pos % 4 == 2:  # 保留75%的帧
            continue 
    elif sparsity == 'Drop90':
        if frame_pos % 10 != 0:  # 保留10%的帧
            continue
    # 'full' 不进行过滤
    
    selected_frames.append(frame_idx)

# 收集选中帧的深度文件（按frame_idx和cam_id排序）
# 注意：frame_groups中已经只包含chosen_cam_id中的相机
depth_files = []
for frame_idx in selected_frames:
    if frame_idx in frame_groups:
        # 确保按frame_idx和cam_id排序
        depth_files.extend(sorted(frame_groups[frame_idx]))

CONSOLE.log(f"[cyan]Sparsity filtering results:[/cyan]")
CONSOLE.log(f"  Sparsity level: {sparsity}")
CONSOLE.log(f"  Total frames: {len(sorted_frame_indices)}")
CONSOLE.log(f"  Selected frames: {len(selected_frames)}")
CONSOLE.log(f"  Selected depth files: {len(depth_files)}")
CONSOLE.log(f"  Expected files per frame: {len(chosen_cam_id)} cameras")

CONSOLE.log("[green]Step 3: Sparsity filtering completed![/green]")


[19:52:52] Sparsity filtering results:                                                                  ]8;id=722790;file:///tmp/ipykernel_96148/1391159818.py\1391159818.py]8;;\:]8;id=81045;file:///tmp/ipykernel_96148/1391159818.py#29\29]8;;\

             Sparsity level: Drop50                                                                     ]8;id=431636;file:///tmp/ipykernel_96148/1391159818.py\1391159818.py]8;;\:]8;id=408886;file:///tmp/ipykernel_96148/1391159818.py#30\30]8;;\

             Total frames: 196                                                                          ]8;id=110389;file:///tmp/ipykernel_96148/1391159818.py\1391159818.py]8;;\:]8;id=701168;file:///tmp/ipykernel_96148/1391159818.py#31\31]8;;\

             Selected frames: 98                                                                        ]8;id=9296;file:///tmp/ipykernel_96148/1391159818.py\1391159818.py]8;;\:]8;id=627138;file:///tmp/ipykernel_96148/1391159818.py#32\32]8;;\

             Selected depth files: 98                                                                   ]8;id=147804;file:///tmp/ipykernel_96148/1391159818.py\1391159818.py]8;;\:]8;id=573053;file:///tmp/ipykernel_96148/1391159818.py#33\33]8;;\

             Expected files per frame: 1 cameras                                                        ]8;id=452143;file:///tmp/ipykernel_96148/1391159818.py\1391159818.py]8;;\:]8;id=990533;file:///tmp/ipykernel_96148/1391159818.py#34\34]8;;\

           Step 3: Sparsity filtering completed!                                                        ]8;id=603551;file:///tmp/ipykernel_96148/1391159818.py\1391159818.py]8;;\:]8;id=909111;file:///tmp/ipykernel_96148/1391159818.py#36\36]8;;\

## 步骤4: 构建Pose映射

构建(frame_idx, cam_id)到pose/intrinsic索引的映射，用于匹配深度文件与poses。


In [10]:
# 读取图像文件列表
images_dir = os.path.join(scene_dir, 'images')
all_image_files = sorted([f for f in os.listdir(images_dir) 
                         if f.endswith('.jpg') or f.endswith('.png')])

# 根据chosen_cam_id过滤图像文件
image_files = []
for img_file in all_image_files:
    try:
        parts = img_file.replace('.jpg', '').replace('.png', '').split('_')
        if len(parts) >= 2:
            cam_id = int(parts[1])
            if cam_id in chosen_cam_id:
                image_files.append(img_file)
    except (ValueError, IndexError):
        continue

CONSOLE.log(f"[cyan]Image file filtering:[/cyan]")
CONSOLE.log(f"  Total image files: {len(all_image_files)}")
CONSOLE.log(f"  Filtered image files (chosen_cam_id={chosen_cam_id}): {len(image_files)}")

# 构建映射: (frame_idx, cam_id) -> index in poses/intrinsics
# 注意：只考虑chosen_cam_id中的相机，并且需要匹配poses/intrinsics的顺序
# poses/intrinsics的顺序: frame0_cam0, frame0_cam1, ..., frame1_cam0, ...
# 但我们需要根据实际深度图来选择对应的poses
pose_map = {}
# 按frame_idx和cam_id排序图像文件
sorted_image_files = sorted(image_files, key=lambda x: (
    int(x.split('_')[0]) if x.split('_')[0].isdigit() else 999999,
    int(x.split('_')[1].split('.')[0]) if len(x.split('_')) > 1 and x.split('_')[1].split('.')[0].isdigit() else 999999
))

for idx, img_file in enumerate(sorted_image_files):
    try:
        # 图像命名: {frame_idx:03d}_{cam_id}.jpg or .png
        parts = img_file.replace('.jpg', '').replace('.png', '').split('_')
        if len(parts) >= 2:
            frame_idx = int(parts[0])
            cam_id = int(parts[1])
            # 只添加chosen_cam_id中的相机
            if cam_id in chosen_cam_id:
                pose_map[(frame_idx, cam_id)] = idx
    except (ValueError, IndexError):
        continue

CONSOLE.log(f"[cyan]Pose mapping statistics:[/cyan]")
CONSOLE.log(f"  Filtered image files: {len(image_files)}")
CONSOLE.log(f"  Pose map entries: {len(pose_map)}")
CONSOLE.log(f"  Expected poses (all cameras): {len(poses)}")
CONSOLE.log(f"  Note: pose_map only includes chosen_cam_id={chosen_cam_id}")

CONSOLE.log("[green]Step 4: Pose mapping completed![/green]")


[19:53:03] Image file filtering:                                                                        ]8;id=510924;file:///tmp/ipykernel_96148/2819835437.py\2819835437.py]8;;\:]8;id=23947;file:///tmp/ipykernel_96148/2819835437.py#18\18]8;;\

             Total image files: 1176                                                                    ]8;id=334863;file:///tmp/ipykernel_96148/2819835437.py\2819835437.py]8;;\:]8;id=455002;file:///tmp/ipykernel_96148/2819835437.py#19\19]8;;\

             Filtered image files (chosen_cam_id=[0]): 196                                              ]8;id=38731;file:///tmp/ipykernel_96148/2819835437.py\2819835437.py]8;;\:]8;id=182760;file:///tmp/ipykernel_96148/2819835437.py#20\20]8;;\

           Pose mapping statistics:                                                                     ]8;id=650190;file:///tmp/ipykernel_96148/2819835437.py\2819835437.py]8;;\:]8;id=71562;file:///tmp/ipykernel_96148/2819835437.py#46\46]8;;\

             Filtered image files: 196                                                                  ]8;id=457930;file:///tmp/ipykernel_96148/2819835437.py\2819835437.py]8;;\:]8;id=446411;file:///tmp/ipykernel_96148/2819835437.py#47\47]8;;\

             Pose map entries: 196                                                                      ]8;id=456266;file:///tmp/ipykernel_96148/2819835437.py\2819835437.py]8;;\:]8;id=104130;file:///tmp/ipykernel_96148/2819835437.py#48\48]8;;\

             Expected poses (all cameras): 120                                                          ]8;id=743784;file:///tmp/ipykernel_96148/2819835437.py\2819835437.py]8;;\:]8;id=105624;file:///tmp/ipykernel_96148/2819835437.py#49\49]8;;\

             Note: pose_map only includes chosen_cam_id=[0]                                             ]8;id=462714;file:///tmp/ipykernel_96148/2819835437.py\2819835437.py]8;;\:]8;id=510584;file:///tmp/ipykernel_96148/2819835437.py#50\50]8;;\

           Step 4: Pose mapping completed!                                                              ]8;id=662137;file:///tmp/ipykernel_96148/2819835437.py\2819835437.py]8;;\:]8;id=239076;file:///tmp/ipykernel_96148/2819835437.py#52\52]8;;\

## 步骤5: 匹配深度文件与Poses

将深度文件与对应的poses和intrinsics匹配，构建c2w和intri列表。


In [11]:
# 匹配深度文件与poses/intrinsics
# 重要：根据深度文件的(frame_idx, cam_id)来查找对应的pose和intrinsic
# 而不是按顺序匹配，因为深度图可能只包含部分相机
valid_depth_files = []
selected_index = []
pcd_generator.c2w = []
pcd_generator.intri = []

for file_name in depth_files:
    try:
        # 从深度文件名解析frame_idx和cam_id
        parts = file_name.replace('.npy', '').split('_')
        if len(parts) >= 2:
            frame_idx = int(parts[0])
            cam_id = int(parts[1])
            
            # 验证相机ID在chosen_cam_id中
            if cam_id not in chosen_cam_id:
                CONSOLE.log(f"Warning: Camera ID {cam_id} not in chosen_cam_id {chosen_cam_id}, skipping {file_name}")
                continue
            
            # 从pose_map中查找对应的pose索引
            if (frame_idx, cam_id) in pose_map:
                idx = pose_map[(frame_idx, cam_id)]
                if idx < len(poses):
                    valid_depth_files.append(file_name)
                    # poses已经对齐并在正确的坐标系中
                    pcd_generator.c2w.append(poses[idx])
                    pcd_generator.intri.append(intrinsics[idx])
                    selected_index.append(idx)
                else:
                    CONSOLE.log(f"Warning: Pose index {idx} out of range for {file_name}")
            else:
                # 如果pose_map中没有，尝试计算索引
                # poses/intrinsics的顺序是: frame0_cam0, frame0_cam1, ..., frame0_camN, frame1_cam0, ...
                # 需要知道总共有多少个相机
                # 尝试从poses数量推断：poses数量 = num_frames * num_cameras
                # 或者从extrinsics目录中推断
                extrinsics_dir = os.path.join(scene_dir, 'extrinsics')
                all_extrinsic_files = [f for f in os.listdir(extrinsics_dir) if f.endswith('.txt')]
                # 获取所有唯一的相机ID
                all_cam_ids = set()
                for f in all_extrinsic_files:
                    try:
                        parts = f.replace('.txt', '').split('_')
                        if len(parts) >= 2:
                            all_cam_ids.add(int(parts[1]))
                    except:
                        continue
                num_cameras = len(all_cam_ids) if all_cam_ids else 6  # 默认6个相机
                
                # 计算索引：frame_idx * num_cameras + cam_id
                # 但需要确保frame_idx在范围内
                calculated_idx = frame_idx * num_cameras + cam_id
                if calculated_idx < len(poses) and calculated_idx >= 0:
                    # 验证这个索引对应的确实是(frame_idx, cam_id)
                    # 通过检查poses的顺序来验证（这里简化处理）
                    valid_depth_files.append(file_name)
                    pcd_generator.c2w.append(poses[calculated_idx])
                    pcd_generator.intri.append(intrinsics[calculated_idx])
                    selected_index.append(calculated_idx)
                    CONSOLE.log(f"  Found pose at calculated index {calculated_idx} for {file_name}")
                else:
                    CONSOLE.log(f"Warning: Cannot find pose for {file_name} (calculated_idx={calculated_idx}, poses_len={len(poses)})")
        else:
            CONSOLE.log(f"Warning: Cannot parse depth filename {file_name}")
    except (ValueError, IndexError) as e:
        CONSOLE.log(f"Warning: Error parsing {file_name}: {e}")

depth_files = valid_depth_files

CONSOLE.log(f"[cyan]Matching results:[/cyan]")
CONSOLE.log(f"  {sparsity}: ALL frames: {len(depth_files)}")
CONSOLE.log(f"  Selected indices: {selected_index[:10]}...")
CONSOLE.log(f"  c2w count: {len(pcd_generator.c2w)}")
CONSOLE.log(f"  intri count: {len(pcd_generator.intri)}")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")

if len(depth_files) == 0:
    raise ValueError("No depth files selected after sparsity filtering and camera filtering")

if len(pcd_generator.c2w) != len(depth_files):
    raise ValueError(f"Mismatch: {len(pcd_generator.c2w)} poses for {len(depth_files)} depth files")

CONSOLE.log("[green]Step 5: Matching completed![/green]")


[19:54:27] Warning: Pose index 120 out of range for 120_0.npy                                           ]8;id=132337;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=433907;file:///tmp/ipykernel_96148/3964186354.py#32\32]8;;\

           Warning: Pose index 121 out of range for 121_0.npy                                           ]8;id=378189;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=267335;file:///tmp/ipykernel_96148/3964186354.py#32\32]8;;\

           Warning: Pose index 124 out of range for 124_0.npy                                           ]8;id=551995;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=102677;file:///tmp/ipykernel_96148/3964186354.py#32\32]8;;\

           Warning: Pose index 125 out of range for 125_0.npy                                           ]8;id=487681;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=73521;file:///tmp/ipykernel_96148/3964186354.py#32\32]8;;\

           Warning: Pose index 128 out of range for 128_0.npy                                           ]8;id=85566;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=729843;file:///tmp/ipykernel_96148/3964186354.py#32\32]8;;\

...

             Selected indices: [0, 1, 4, 5, 8, 9, 12, 13, 16, 17]...                                    ]8;id=501535;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=628649;file:///tmp/ipykernel_96148/3964186354.py#74\74]8;;\

             c2w count: 60                                                                              ]8;id=628700;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=688712;file:///tmp/ipykernel_96148/3964186354.py#75\75]8;;\

             intri count: 60                                                                            ]8;id=59;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=290284;file:///tmp/ipykernel_96148/3964186354.py#76\76]8;;\

             Chosen camera IDs: [0]                                                                     ]8;id=885367;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=765736;file:///tmp/ipykernel_96148/3964186354.py#77\77]8;;\

           Step 5: Matching completed!                                                                  ]8;id=388639;file:///tmp/ipykernel_96148/3964186354.py\3964186354.py]8;;\:]8;id=429628;file:///tmp/ipykernel_96148/3964186354.py#85\85]8;;\

## 步骤6: 深度一致性检查

检查连续帧之间的深度一致性，生成一致性掩码。


In [12]:
# 深度一致性检查
if depth_consistency:
    CONSOLE.log("[cyan]Performing depth consistency check...[/cyan]")
    consistency_mask = pcd_generator.depth_cosistency_check(depth_files=depth_files, H=H, W=W)
    CONSOLE.log(f"[green]Generated {len(consistency_mask)} consistency masks[/green]")
else:
    consistency_mask = [np.ones((H, W)) for _ in range(len(depth_files))]
    CONSOLE.log("[yellow]Depth consistency check disabled[/yellow]")

# 显示一致性掩码统计
if len(consistency_mask) > 0:
    avg_valid_ratio = np.mean([mask.sum() / mask.size for mask in consistency_mask])
    CONSOLE.log(f"[cyan]Consistency mask statistics:[/cyan]")
    CONSOLE.log(f"  Average valid ratio: {avg_valid_ratio:.2%}")

CONSOLE.log("[green]Step 6: Depth consistency check completed![/green]")


[19:54:40] Performing depth consistency check...                                                         ]8;id=205196;file:///tmp/ipykernel_96148/1612303207.py\1612303207.py]8;;\:]8;id=892356;file:///tmp/ipykernel_96148/1612303207.py#3\3]8;;\

Depth Consistency Check!


/tmp/ipykernel_96148/3590799288.py:111: RuntimeWarning: invalid value encountered in cast
  last_pixels = np.vstack((last_x, last_y)).T.reshape(-1, 2).astype(np.int32)
/tmp/ipykernel_96148/3590799288.py:130: RuntimeWarning: Mean of empty slice.
  depth_mask[valid_mask] = np.where(depth_diff < depth_diff.mean(), 1, 0)
/root/miniconda3/envs/metric3d/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


[19:54:50] Generated 60 consistency masks                                                                ]8;id=708085;file:///tmp/ipykernel_96148/1612303207.py\1612303207.py]8;;\:]8;id=192260;file:///tmp/ipykernel_96148/1612303207.py#5\5]8;;\

           Consistency mask statistics:                                                                 ]8;id=952567;file:///tmp/ipykernel_96148/1612303207.py\1612303207.py]8;;\:]8;id=933145;file:///tmp/ipykernel_96148/1612303207.py#13\13]8;;\

             Average valid ratio: 87.13%                                                                ]8;id=74302;file:///tmp/ipykernel_96148/1612303207.py\1612303207.py]8;;\:]8;id=805050;file:///tmp/ipykernel_96148/1612303207.py#14\14]8;;\

           Step 6: Depth consistency check completed!                                                   ]8;id=979504;file:///tmp/ipykernel_96148/1612303207.py\1612303207.py]8;;\:]8;id=449978;file:///tmp/ipykernel_96148/1612303207.py#16\16]8;;\

## 步骤7: 点云累积主循环

这是核心步骤，将深度图转换为点云。包括：
- 加载和预处理深度图（使用depth_utils）
- 加载RGB图像
- 应用各种掩码（天空、一致性、下采样）
- 反投影到相机坐标系
- 变换到世界坐标系
- 累积点云


In [13]:
# 初始化下采样掩码
color_pointclouds = []
if downscale != 1:
    downscale_mask = np.zeros_like(consistency_mask[0])
    downscale_mask[::downscale, ::downscale] = 1
    CONSOLE.log(f"[cyan]Downscale mask created: sampling every {downscale} pixels[/cyan]")
else:
    downscale_mask = None

CONSOLE.log(f"[cyan]Starting point cloud accumulation for {len(depth_files)} depth files...[/cyan]")

# 遍历所有深度文件
for i, file_name in enumerate(depth_files):
    if (i + 1) % 10 == 0 or i == 0:
        CONSOLE.log(f"Processing {i+1}/{len(depth_files)}: {file_name}")
    
    # 7.1: 加载和预处理深度图（使用depth_utils）
    depth_file = os.path.join(pcd_generator.depth_dir, file_name)
    try:
        # 重要：使用depth_utils进行预处理，自动插值到原始尺寸
        depth, metadata = process_depth_for_use(depth_file, target_shape=(H, W))
    except Exception as e:
        CONSOLE.log(f"Warning: Failed to load depth {file_name}: {e}")
        continue
    
    # 7.2: 加载RGB图像
    rgb_file = os.path.join(scene_dir, 'images', file_name.replace('.npy', '.jpg'))
    if not os.path.exists(rgb_file):
        rgb_file = os.path.join(scene_dir, 'images', file_name.replace('.npy', '.png'))
    
    if not os.path.exists(rgb_file):
        CONSOLE.log(f"Warning: RGB file not found for {file_name}")
        continue
    
    try:
        rgb = imageio.imread(rgb_file) / 255.0
    except Exception as e:
        CONSOLE.log(f"Warning: Failed to load RGB {file_name}: {e}")
        continue
    
    # 7.3: 应用天空过滤（如果启用）
    if filter_sky:
        sky_mask_file = os.path.join(
            scene_dir, 'sky_masks', 
            file_name.replace('.npy', '.png')
        )
        if os.path.exists(sky_mask_file):
            sky_mask = cv2.imread(sky_mask_file, cv2.IMREAD_GRAYSCALE)
            # sky_masks: 0 = sky, 255 = non-sky
            # Convert to boolean: True = non-sky (keep), False = sky (filter)
            mask = (sky_mask > 0).astype(np.bool_)
            final_mask = np.logical_and(consistency_mask[i], mask)
        else:
            CONSOLE.log(f"Warning: Sky mask not found: {sky_mask_file}, skipping sky filtering")
            final_mask = consistency_mask[i]
    else:
        final_mask = consistency_mask[i]
    
    # 7.4: 应用下采样掩码
    if downscale != 1:
        final_mask = np.logical_and(downscale_mask, final_mask)
    
    # 7.5: 提取有效像素
    kept = np.argwhere(final_mask)
    
    if len(kept) == 0:
        continue
    
    depth_values = depth[kept[:, 0], kept[:, 1]]
    rgb_values = rgb[kept[:, 0], kept[:, 1]]
    
    # 7.6: 过滤无效深度值
    valid_depth_mask = np.isfinite(depth_values) & (depth_values > 0)
    if not np.any(valid_depth_mask):
        continue
    
    depth_values = depth_values[valid_depth_mask]
    rgb_values = rgb_values[valid_depth_mask]
    kept_valid = kept[valid_depth_mask]
    
    try:
        c2w = pcd_generator.c2w[i]
        K = pcd_generator.intri[i]
    except IndexError as e:
        CONSOLE.log(f"Warning: Index error for {file_name}: {e}")
        continue
    
    # 7.7: 反投影到相机坐标系
    # 生成像素坐标网格
    x = np.arange(0, W)
    y = np.arange(0, H)
    xx, yy = np.meshgrid(x, y)
    pixels = np.vstack((xx.ravel(), yy.ravel())).T.reshape(H, W, 2)
    
    # 反投影深度到3D点
    pixel_coords = pixels[kept_valid[:, 0], kept_valid[:, 1]]
    x_cam = (pixel_coords[:, 0] - K[0, 2]) * depth_values / K[0, 0]
    y_cam = (pixel_coords[:, 1] - K[1, 2]) * depth_values / K[1, 1]
    z_cam = depth_values
    coordinates = np.stack([x_cam, y_cam, z_cam], axis=1)
    
    # 过滤NaN/inf坐标
    valid_coords_mask = np.isfinite(coordinates).all(axis=1)
    if not np.any(valid_coords_mask):
        continue
    
    coordinates = coordinates[valid_coords_mask]
    rgb_values = rgb_values[valid_coords_mask]
    coordinates = np.column_stack((coordinates, np.ones(len(coordinates))))
    
    # 7.8: 变换到世界坐标系
    worlds = np.dot(c2w, coordinates.T).T
    worlds = worlds[:, :3]
    
    # 过滤NaN/inf世界坐标
    valid_worlds_mask = np.isfinite(worlds).all(axis=1)
    if not np.any(valid_worlds_mask):
        continue
    
    worlds = worlds[valid_worlds_mask]
    rgb_values = rgb_values[valid_worlds_mask]
    
    # 7.9: 累积点云块
    point_cloud_chunk = np.concatenate([worlds, rgb_values.reshape(-1, 3)], axis=-1)
    color_pointclouds.append(point_cloud_chunk)

# 7.10: 合并所有点云块
if len(color_pointclouds) == 0:
    raise ValueError("No valid point cloud generated")

accumulated_pointcloud = np.concatenate(color_pointclouds, axis=0).reshape(-1, 6)

# 最终过滤：移除剩余的NaN/inf值
valid_mask = np.isfinite(accumulated_pointcloud[:, :3]).all(axis=1)
if not np.any(valid_mask):
    raise ValueError("No valid point cloud after NaN filtering")

accumulated_pointcloud = accumulated_pointcloud[valid_mask]

CONSOLE.log(f"[green]Step 7: Point cloud accumulation completed![/green]")
CONSOLE.log(f"  Total points: {len(accumulated_pointcloud)}")
CONSOLE.log(f"  Point cloud shape: {accumulated_pointcloud.shape}")


[19:55:59] Downscale mask created: sampling every 2 pixels                                               ]8;id=693058;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=538407;file:///tmp/ipykernel_96148/3153704795.py#6\6]8;;\

           Starting point cloud accumulation for 60 depth files...                                      ]8;id=43226;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=433131;file:///tmp/ipykernel_96148/3153704795.py#10\10]8;;\

           Processing 1/60: 000_0.npy                                                                   ]8;id=530944;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=998176;file:///tmp/ipykernel_96148/3153704795.py#15\15]8;;\

[19:56:00] Processing 10/60: 017_0.npy                                                                  ]8;id=512078;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=610067;file:///tmp/ipykernel_96148/3153704795.py#15\15]8;;\

[19:56:01] Processing 20/60: 037_0.npy                                                                  ]8;id=487138;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=781331;file:///tmp/ipykernel_96148/3153704795.py#15\15]8;;\

...

[19:56:05] Processing 50/60: 097_0.npy                                                                  ]8;id=974422;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=236371;file:///tmp/ipykernel_96148/3153704795.py#15\15]8;;\

[19:56:06] Processing 60/60: 117_0.npy                                                                  ]8;id=725304;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=230650;file:///tmp/ipykernel_96148/3153704795.py#15\15]8;;\

[19:56:08] Step 7: Point cloud accumulation completed!                                                 ]8;id=995307;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=643126;file:///tmp/ipykernel_96148/3153704795.py#140\140]8;;\

             Total points: 17341526                                                                    ]8;id=63144;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=269554;file:///tmp/ipykernel_96148/3153704795.py#141\141]8;;\

             Point cloud shape: (17341526, 6)                                                          ]8;id=24703;file:///tmp/ipykernel_96148/3153704795.py\3153704795.py]8;;\:]8;id=669824;file:///tmp/ipykernel_96148/3153704795.py#142\142]8;;\

In [14]:
points = accumulated_pointcloud[:, :3]
colors = accumulated_pointcloud[:, 3:]

if use_bbx:
    bbx_min, bbx_max = pcd_generator.get_bbx()
    CONSOLE.log(f"[cyan]Bounding box range:[/cyan]")
    CONSOLE.log(f"  Min: {bbx_min}")
    CONSOLE.log(f"  Max: {bbx_max}")
    
    # 裁剪点云
    points, colors = pcd_generator.crop_pointcloud(bbx_min, bbx_max, points, colors)
    
    # 分离内部和外部点云
    inside_pnt, inside_rgb, outside_pnt, outside_rgb = pcd_generator.split_pointcloud(
        bbx_min, bbx_max, points, colors
    )
    
    CONSOLE.log(f"[cyan]Bounding box filtering results:[/cyan]")
    CONSOLE.log(f"  Inside points: {len(inside_pnt)}")
    CONSOLE.log(f"  Outside points: {len(outside_pnt)}")
    CONSOLE.log(f"  Total after crop: {len(points)}")
else:
    CONSOLE.log("[yellow]Bounding box filtering disabled[/yellow]")
    inside_pnt, inside_rgb, outside_pnt, outside_rgb = None, None, None, None

CONSOLE.log("[green]Step 8: Bounding box cropping completed![/green]")


[19:56:26] Bounding box range:                                                                           ]8;id=843439;file:///tmp/ipykernel_96148/3908234961.py\3908234961.py]8;;\:]8;id=569962;file:///tmp/ipykernel_96148/3908234961.py#6\6]8;;\

             Min: [-20 -20 -20]                                                                          ]8;id=137650;file:///tmp/ipykernel_96148/3908234961.py\3908234961.py]8;;\:]8;id=616080;file:///tmp/ipykernel_96148/3908234961.py#7\7]8;;\

             Max: [20.   4.8 70. ]                                                                       ]8;id=856742;file:///tmp/ipykernel_96148/3908234961.py\3908234961.py]8;;\:]8;id=30113;file:///tmp/ipykernel_96148/3908234961.py#8\8]8;;\

[19:56:28] Bounding box filtering results:                                                              ]8;id=587504;file:///tmp/ipykernel_96148/3908234961.py\3908234961.py]8;;\:]8;id=309343;file:///tmp/ipykernel_96148/3908234961.py#18\18]8;;\

             Inside points: 11058676                                                                    ]8;id=728496;file:///tmp/ipykernel_96148/3908234961.py\3908234961.py]8;;\:]8;id=145869;file:///tmp/ipykernel_96148/3908234961.py#19\19]8;;\

             Outside points: 62884                                                                      ]8;id=147079;file:///tmp/ipykernel_96148/3908234961.py\3908234961.py]8;;\:]8;id=908789;file:///tmp/ipykernel_96148/3908234961.py#20\20]8;;\

             Total after crop: 11121560                                                                 ]8;id=953510;file:///tmp/ipykernel_96148/3908234961.py\3908234961.py]8;;\:]8;id=320949;file:///tmp/ipykernel_96148/3908234961.py#21\21]8;;\

           Step 8: Bounding box cropping completed!                                                     ]8;id=6760;file:///tmp/ipykernel_96148/3908234961.py\3908234961.py]8;;\:]8;id=742518;file:///tmp/ipykernel_96148/3908234961.py#26\26]8;;\

## 步骤9: 点云滤波

对点云进行统计滤波和均匀下采样，去除噪声。


In [15]:
if use_bbx:
    # 内部点云滤波
    inside_pointcloud = o3d.geometry.PointCloud()
    inside_pointcloud.points = o3d.utility.Vector3dVector(inside_pnt[:, :3])
    inside_pointcloud.colors = o3d.utility.Vector3dVector(inside_rgb)
    
    CONSOLE.log("[cyan]Filtering inside point cloud...[/cyan]")
    cl, ind = inside_pointcloud.remove_statistical_outlier(
        nb_neighbors=35, std_ratio=1.5
    )
    inside_pointcloud = inside_pointcloud.select_by_index(ind)
    CONSOLE.log(f"  Inside points after filtering: {len(inside_pointcloud.points)}")
    
    # 外部点云滤波
    outside_pointcloud = o3d.geometry.PointCloud()
    outside_pointcloud.points = o3d.utility.Vector3dVector(outside_pnt[:, :3])
    outside_pointcloud.colors = o3d.utility.Vector3dVector(outside_rgb)
    
    CONSOLE.log("[cyan]Filtering outside point cloud...[/cyan]")
    cl, ind = outside_pointcloud.remove_statistical_outlier(
        nb_neighbors=20, std_ratio=2.0
    )
    outside_pointcloud = outside_pointcloud.select_by_index(ind)
    CONSOLE.log(f"  Outside points after filtering: {len(outside_pointcloud.points)}")
    
    # 合并并下采样
    combined_pointcloud = inside_pointcloud + outside_pointcloud
    combined_pointcloud = combined_pointcloud.uniform_down_sample(every_k_points=2)
    final_pointcloud = combined_pointcloud
    
    CONSOLE.log(f"  Final points after downsampling: {len(final_pointcloud.points)}")
else:
    # 全局滤波
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points[:, :3])
    point_cloud.colors = o3d.utility.Vector3dVector(colors)
    
    CONSOLE.log("[cyan]Filtering point cloud...[/cyan]")
    cl, ind = point_cloud.remove_statistical_outlier(
        nb_neighbors=30, std_ratio=1.5
    )
    point_cloud = point_cloud.select_by_index(ind)
    point_cloud = point_cloud.uniform_down_sample(every_k_points=3)
    final_pointcloud = point_cloud
    
    CONSOLE.log(f"  Final points: {len(final_pointcloud.points)}")

CONSOLE.log("[green]Step 9: Point cloud filtering completed![/green]")


[19:56:36] Filtering inside point cloud...                                                               ]8;id=138113;file:///tmp/ipykernel_96148/2555259510.py\2555259510.py]8;;\:]8;id=529199;file:///tmp/ipykernel_96148/2555259510.py#7\7]8;;\

[19:56:46]   Inside points after filtering: 10521133                                                    ]8;id=359580;file:///tmp/ipykernel_96148/2555259510.py\2555259510.py]8;;\:]8;id=165704;file:///tmp/ipykernel_96148/2555259510.py#12\12]8;;\

           Filtering outside point cloud...                                                             ]8;id=939615;file:///tmp/ipykernel_96148/2555259510.py\2555259510.py]8;;\:]8;id=152080;file:///tmp/ipykernel_96148/2555259510.py#19\19]8;;\

             Outside points after filtering: 60115                                                      ]8;id=993474;file:///tmp/ipykernel_96148/2555259510.py\2555259510.py]8;;\:]8;id=806296;file:///tmp/ipykernel_96148/2555259510.py#24\24]8;;\

[19:56:47]   Final points after downsampling: 5290624                                                   ]8;id=986996;file:///tmp/ipykernel_96148/2555259510.py\2555259510.py]8;;\:]8;id=72485;file:///tmp/ipykernel_96148/2555259510.py#31\31]8;;\

           Step 9: Point cloud filtering completed!                                                     ]8;id=407327;file:///tmp/ipykernel_96148/2555259510.py\2555259510.py]8;;\:]8;id=456002;file:///tmp/ipykernel_96148/2555259510.py#48\48]8;;\

## 步骤10: 保存点云

将处理后的点云保存为PLY格式。


In [16]:
# 创建保存目录
save_path = os.path.join(scene_dir, save_dir_name)
os.makedirs(save_path, exist_ok=True)

# 保存点云
output_file = os.path.join(save_path, f'{frame_start}.ply')
try:
    o3d.io.write_point_cloud(output_file, final_pointcloud)
    CONSOLE.log(f"[green]Point cloud saved successfully![/green]")
    CONSOLE.log(f"  Output file: {output_file}")
    CONSOLE.log(f"  Point count: {len(final_pointcloud.points)}")
except Exception as e:
    CONSOLE.log(f"[red]Error saving point cloud: {e}[/red]")
    raise

CONSOLE.log("[green]Step 10: Point cloud saving completed![/green]")


[19:58:56] Point cloud saved successfully!                                                               ]8;id=288876;file:///tmp/ipykernel_96148/3136482562.py\3136482562.py]8;;\:]8;id=819262;file:///tmp/ipykernel_96148/3136482562.py#9\9]8;;\

             Output file: /root/autodl-tmp/nuScenes/preprocess/trainval/000/Drop50/0.ply                ]8;id=15818;file:///tmp/ipykernel_96148/3136482562.py\3136482562.py]8;;\:]8;id=138049;file:///tmp/ipykernel_96148/3136482562.py#10\10]8;;\

             Point count: 5290624                                                                       ]8;id=16527;file:///tmp/ipykernel_96148/3136482562.py\3136482562.py]8;;\:]8;id=319657;file:///tmp/ipykernel_96148/3136482562.py#11\11]8;;\

           Step 10: Point cloud saving completed!                                                       ]8;id=792655;file:///tmp/ipykernel_96148/3136482562.py\3136482562.py]8;;\:]8;id=999943;file:///tmp/ipykernel_96148/3136482562.py#16\16]8;;\

## 反直觉检查

检查可能的问题和潜在错误。


In [ ]:
CONSOLE.log("[cyan]Performing counterintuitive checks...[/cyan]")

# 1. 检查深度图尺寸
CONSOLE.log("\n[cyan]1. Depth map size check:[/cyan]")
sample_depth_file = os.path.join(pcd_generator.depth_dir, depth_files[0])
try:
    depth_raw, metadata = load_depth_with_metadata(sample_depth_file)
    depth_processed, _ = process_depth_for_use(sample_depth_file, target_shape=(H, W))
    CONSOLE.log(f"  Raw depth shape: {depth_raw.shape}")
    CONSOLE.log(f"  Processed depth shape: {depth_processed.shape}")
    CONSOLE.log(f"  Target shape: ({H}, {W})")
    CONSOLE.log(f"  Metadata ori_shape: {metadata.get('ori_shape', 'N/A')}")
    if depth_raw.shape != (H, W):
        CONSOLE.log(f"  [yellow]⚠ Depth map needs interpolation (raw != target)[/yellow]")
    else:
        CONSOLE.log(f"  [green]✓ Depth map size matches target[/green]")
except Exception as e:
    CONSOLE.log(f"  [red]✗ Error checking depth size: {e}[/red]")

# 2. 检查文件名解析和相机ID
CONSOLE.log("\n[cyan]2. Filename parsing and camera ID check:[/cyan]")
sample_files = depth_files[:5]
found_cam_ids = set()
for f in sample_files:
    try:
        parts = f.replace('.npy', '').split('_')
        if len(parts) >= 2:
            frame_idx = int(parts[0])
            cam_id = int(parts[1])
            found_cam_ids.add(cam_id)
            in_chosen = "✓" if cam_id in chosen_cam_id else "✗"
            CONSOLE.log(f"  {f} -> frame_idx={frame_idx}, cam_id={cam_id} {in_chosen}")
        else:
            CONSOLE.log(f"  [red]✗ Cannot parse: {f}[/red]")
    except Exception as e:
        CONSOLE.log(f"  [red]✗ Error parsing {f}: {e}[/red]")

# 检查所有深度文件的相机ID是否都在chosen_cam_id中
all_depth_cam_ids = set()
for f in depth_files:
    try:
        parts = f.replace('.npy', '').split('_')
        if len(parts) >= 2:
            all_depth_cam_ids.add(int(parts[1]))
    except:
        continue

if all_depth_cam_ids.issubset(set(chosen_cam_id)):
    CONSOLE.log(f"  [green]✓ All depth file camera IDs ({all_depth_cam_ids}) are in chosen_cam_id ({chosen_cam_id})[/green]")
else:
    unexpected = all_depth_cam_ids - set(chosen_cam_id)
    CONSOLE.log(f"  [red]✗ Found unexpected camera IDs in depth files: {unexpected}[/red]")
    CONSOLE.log(f"  [yellow]⚠ This may cause matching issues. Check chosen_cam_id configuration.[/yellow]")

# 3. 检查数量匹配
CONSOLE.log("\n[cyan]3. Count matching check:[/cyan]")
CONSOLE.log(f"  Poses count: {len(poses)}")
CONSOLE.log(f"  Intrinsics count: {len(intrinsics)}")
CONSOLE.log(f"  Depth files count: {len(depth_files)}")
CONSOLE.log(f"  c2w count: {len(pcd_generator.c2w)}")
CONSOLE.log(f"  intri count: {len(pcd_generator.intri)}")
if len(pcd_generator.c2w) == len(depth_files) == len(pcd_generator.intri):
    CONSOLE.log(f"  [green]✓ All counts match[/green]")
else:
    CONSOLE.log(f"  [red]✗ Count mismatch detected![/red]")

# 4. 检查深度值范围
CONSOLE.log("\n[cyan]4. Depth value range check:[/cyan]")
try:
    sample_depth, _ = process_depth_for_use(sample_depth_file, target_shape=(H, W))
    valid_depths = sample_depth[sample_depth > 0]
    if len(valid_depths) > 0:
        CONSOLE.log(f"  Valid depth range: [{valid_depths.min():.2f}, {valid_depths.max():.2f}] meters")
        CONSOLE.log(f"  Mean depth: {valid_depths.mean():.2f} meters")
        if valid_depths.max() > 200:
            CONSOLE.log(f"  [yellow]⚠ Some depth values exceed 200m (may be outliers)[/yellow]")
        else:
            CONSOLE.log(f"  [green]✓ Depth values in reasonable range[/green]")
    else:
        CONSOLE.log(f"  [red]✗ No valid depth values found[/red]")
except Exception as e:
    CONSOLE.log(f"  [red]✗ Error checking depth range: {e}[/red]")

# 5. 检查坐标变换
CONSOLE.log("\n[cyan]5. Coordinate transformation check:[/cyan]")
sample_c2w = pcd_generator.c2w[0]
CONSOLE.log(f"  Sample c2w matrix shape: {sample_c2w.shape}")
CONSOLE.log(f"  Determinant: {np.linalg.det(sample_c2w[:3, :3]):.6f}")
if abs(np.linalg.det(sample_c2w[:3, :3]) - 1.0) < 0.01:
    CONSOLE.log(f"  [green]✓ Rotation matrix is valid (det ≈ 1)[/green]")
else:
    CONSOLE.log(f"  [yellow]⚠ Rotation matrix determinant is not 1[/yellow]")

# 6. 检查掩码应用顺序
CONSOLE.log("\n[cyan]6. Mask application order check:[/cyan]")
CONSOLE.log(f"  Order: consistency -> sky -> downscale")
CONSOLE.log(f"  [green]✓ Correct order maintained[/green]")

# 7. 检查深度图元数据
CONSOLE.log("\n[cyan]7. Depth metadata check:[/cyan]")
meta_file = sample_depth_file.replace('.npy', '_meta.npz')
if os.path.exists(meta_file):
    meta = dict(np.load(meta_file))
    CONSOLE.log(f"  [green]✓ Metadata file exists[/green]")
    CONSOLE.log(f"  Keys: {list(meta.keys())}")
else:
    CONSOLE.log(f"  [yellow]⚠ Metadata file not found: {meta_file}[/yellow]")

# 8. 检查内参格式
CONSOLE.log("\n[cyan]8. Intrinsic format check:[/cyan]")
sample_K = pcd_generator.intri[0]
CONSOLE.log(f"  Intrinsic matrix shape: {sample_K.shape}")
CONSOLE.log(f"  fx={sample_K[0,0]:.2f}, fy={sample_K[1,1]:.2f}")
CONSOLE.log(f"  cx={sample_K[0,2]:.2f}, cy={sample_K[1,2]:.2f}")
if sample_K.shape == (4, 4):
    CONSOLE.log(f"  [green]✓ Intrinsic is 4x4 matrix format[/green]")
else:
    CONSOLE.log(f"  [yellow]⚠ Intrinsic is not 4x4 format[/yellow]")

CONSOLE.log("\n[green]Counterintuitive checks completed![/green]")


In [17]:
# 验证输出文件
if os.path.exists(output_file):
    CONSOLE.log(f"[green]✓ Output file exists: {output_file}[/green]")
    
    # 读取点云以验证
    loaded_pcd = o3d.io.read_point_cloud(output_file)
    
    CONSOLE.log(f"[cyan]Point cloud statistics:[/cyan]")
    CONSOLE.log(f"  Point count: {len(loaded_pcd.points)}")
    
    if len(loaded_pcd.points) > 0:
        points_array = np.asarray(loaded_pcd.points)
        CONSOLE.log(f"  X range: [{points_array[:, 0].min():.2f}, {points_array[:, 0].max():.2f}]")
        CONSOLE.log(f"  Y range: [{points_array[:, 1].min():.2f}, {points_array[:, 1].max():.2f}]")
        CONSOLE.log(f"  Z range: [{points_array[:, 2].min():.2f}, {points_array[:, 2].max():.2f}]")
        
        if loaded_pcd.has_colors():
            CONSOLE.log(f"  [green]✓ Point cloud has colors[/green]")
        else:
            CONSOLE.log(f"  [yellow]⚠ Point cloud has no colors[/yellow]")
        
        # 文件大小
        file_size = os.path.getsize(output_file) / (1024 * 1024)  # MB
        CONSOLE.log(f"  File size: {file_size:.2f} MB")
    else:
        CONSOLE.log(f"  [red]✗ Point cloud is empty![/red]")
else:
    CONSOLE.log(f"[red]✗ Output file not found: {output_file}[/red]")

CONSOLE.log("\n[green]Point cloud generation pipeline completed successfully![/green]")
CONSOLE.log(f"  Output: {output_file}")


[19:59:18] ✓ Output file exists: /root/autodl-tmp/nuScenes/preprocess/trainval/000/Drop50/0.ply           ]8;id=598300;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=529507;file:///tmp/ipykernel_96148/416561032.py#3\3]8;;\

[19:59:19] Point cloud statistics:                                                                        ]8;id=360023;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=439475;file:///tmp/ipykernel_96148/416561032.py#8\8]8;;\

             Point count: 5290624                                                                         ]8;id=841764;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=373105;file:///tmp/ipykernel_96148/416561032.py#9\9]8;;\

             X range: [-20.00, 20.00]                                                                    ]8;id=52224;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=532913;file:///tmp/ipykernel_96148/416561032.py#13\13]8;;\

             Y range: [-20.00, 3.07]                                                                     ]8;id=171025;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=630630;file:///tmp/ipykernel_96148/416561032.py#14\14]8;;\

             Z range: [-20.00, 119.99]                                                                   ]8;id=588776;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=362695;file:///tmp/ipykernel_96148/416561032.py#15\15]8;;\

             ✓ Point cloud has colors                                                                    ]8;id=120878;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=493601;file:///tmp/ipykernel_96148/416561032.py#18\18]8;;\

             File size: 136.23 MB                                                                        ]8;id=129261;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=751099;file:///tmp/ipykernel_96148/416561032.py#24\24]8;;\

                                                                                                         ]8;id=488681;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=878310;file:///tmp/ipykernel_96148/416561032.py#30\30]8;;\
           Point cloud generation pipeline completed successfully!                                                      

             Output: /root/autodl-tmp/nuScenes/preprocess/trainval/000/Drop50/0.ply                      ]8;id=39809;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=187137;file:///tmp/ipykernel_96148/416561032.py#31\31]8;;\